# Implementacion el metodo de la maxima verosimilitud para la regresion logistica

### Definir la funcion de entorno L(b)

In [1]:
from IPython.display import display, Math, Latex
formula = r'L(\beta)=\sum_{i=1}^n P_i^{y_i}({1-pi})^{y_i}'
display(Math(formula))



<IPython.core.display.Math object>

In [2]:
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1,len(y) + 1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1,pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

### Calcular las probabilidades para cada observacion

In [3]:
formula = r'P_i = P(x_i) = \frac{1}{1+e^{-\beta\cdot x_i}}'
display(Math(formula))

<IPython.core.display.Math object>

In [4]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi = list(range(1,n_rows + 1))
    expon = list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore",invalid="ignore"):
            pi[i] = 1/ (1 + np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

In [5]:
formula = r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'
display(Math(formula))

<IPython.core.display.Math object>

In [6]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n * n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i] = pi[i] * (1 - pi[i])
        W[i,i].astype(float)
    return W

### Obtener la solucion de la funcion logistica

In [7]:
formula = r"\beta_{n + 1} = \beta_n -\frac{f(x_n)}{f'(x_n)}"
display(Math(formula))
formula = r"f(\beta) = X(Y-P)"
display(Math(formula))
formula = r"f'(\beta) = XWX^T"
display(Math(formula))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
def logistics(X,Y,limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X,bias,axis=1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol + 1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i > limit):
        print("Iter:i " + str(iter_i) + "," + str(limit))
        pi = logitprobs(X_new,beta)
        print("Pi: " + str(pi))
        W = findW(pi)
        print("W: " + str(W))
        num = (np.transpose(np.matrix(X_new)) * np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new)) * np.matrix(W) * np.matrix(X_new))
        #inc = np.array(linalg.inv(den) * num)
        root_dif = np.array(linalg.inv(den) * num)
        beta = beta + root_dif
        print("Beta: " + str(beta))
        iter_i = np.sum(root_dif * root_dif)
        ll = likelihood(Y,pi)
        print("********************************************************************************")
    return beta
        
        

## Comprobacion experimental

In [9]:
import numpy as np

In [10]:
X = np.array(range(10)).reshape(10,1)

In [11]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [12]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [13]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias,axis=1)

In [14]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [15]:
a = logistics(X,Y,0.00001)

Iter:i 10000,1e-05
Pi: [array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W: [[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
********************************************************************************
Iter:i 5.777190082644626,1e-05
Pi: [array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), arr

In [16]:
ll = likelihood(Y, logitprobs(X,a))

In [17]:
ll

array([1.32622426e-06])

In [18]:
#Y = 0.66220827 * X - 3.69557172

## Con el paquete statsmodel de python


In [19]:
import statsmodels.api as sm

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [20]:
X_new = np.append(X,bias,axis=1)
logit_model = sm.Logit(Y, X_new)

In [21]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [22]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2020-05-14 16:53 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906
===============================================================

"""

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])